In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MAX_obj:                    3245
ZAMOWIONE_obj:              1052
CZEKA_NA_SPAKOWANIE_obj:    102
SALDO_obj:                  2081
WOLNE_obj:                  1865
WOLNE_NIE_SPAK_obj:         1967
DO_ZAM_obj:                 434
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 64.1%
WOLNE / MAX: 57.5%
WOLNE_NIE_SPAK / MAX: 60.6%
ZAPEŁNIENIE MAG PO ZDJECIU 1 PACZEK RAZEM Z ZAMOWIENIAMI: 93.0%


In [3]:
Podsumowanie_paczek_i_Pw("24/61")

OBIĘTOSC PACZEK (Z OWATAMI)
27/01: 164m3
WST: 53m3
Objetość wszystkich paczek: 216m3
PW 24/61 spakowano: 309M3
----
       PW              KOD                                OPIS  ILOSC  \
0   24/61  16.130.00001.60           WILLOW [ - KOMPLET PIANEK     70   
1   24/61  16.130.00002.60           WILLOW ] - KOMPLET PIANEK     70   
2   24/61  16.130.65000.60       WILLOW [L2Hz - KOMPLET PIANEK     20   
3   24/61  16.136.10003.60          KELLY [1] - KOMPLET PIANEK     20   
4   24/61  16.115.63001.60            RITZ [L - KOMPLET PIANEK      5   
5   24/61  16.129.65001.60  HUDSON [LH 438215 - KOMPLET PIANEK     10   
6   24/61  16.117.35003.60         DIVA X3,5X - KOMPLET PIANEK     40   
7   24/61  16.117.40003.60           DIVA X4X - KOMPLET PIANEK     30   
8   24/61  16.117.25003.60         DIVA X2,5X - KOMPLET PIANEK     20   
9   24/61  16.009.25000.60          AVANT 2,5 - KOMPLET PIANEK     10   
10  24/61  16.009.30000.60            AVANT 3 - KOMPLET PIANEK     10   
11  24/6

In [5]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
br.to_excel("BRAKI.xlsx")

In [5]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 95 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 19 POZYCJE
SALDO PONIZEJ MIN: 84 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 17 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
13,AVANT 2][,0.0,0.0,0.0,7,0.0
45,CALYPSO 1,0.0,0.0,11.0,18,9.0
53,CALYPSO [L2H,0.0,0.0,6.0,8,6.0
59,DIVA X1X,0.0,0.0,2.0,3,1.0
68,DIVA 4X,0.0,0.0,3.0,4,3.0
73,DIVA P113x95,0.0,0.0,1.0,2,1.0
74,DIVA P95x95,0.0,0.0,7.0,8,7.0
94,"EXTREME 1,5s",0.0,0.0,0.0,8,0.0
97,"EXTREME [1,5s",0.0,0.0,0.0,8,0.0
107,EXTREME P80x105,0.0,0.0,0.0,8,0.0


In [2]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)



In [ ]:
arp["WILLOW"].ar

In [ ]:
(arp > 1).query("GRUPA < 4")

In [ ]:
arp["ELIXIR"].Wykres_obj()

In [ ]:
arp.Tabela_podsumowania_analizy#.query("GRUPA == 4")

In [ ]:
arp["STONE"].ar

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [3]:
Zapis_danych_do_Archiwum(243m, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")